In [1]:
import theano
print(theano.__version__)

0.8.0.dev-9d0c52b9254c0d30dacbf1e2a231112ee3e3b99a


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


In [2]:
import os
import numpy as np

from scipy import ndimage
from scipy.misc import imresize

from sklearn.utils import shuffle
from sklearn.cross_validation import StratifiedKFold
from keras.utils import np_utils

from sklearn.feature_extraction.image import extract_patches_2d

directory = './Images/'
# input image dimensions
# img_rows, img_cols = 1024, 1024
img_rows, img_cols = 128, 128

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

patch_count = 200

def load(seed=123):
    X = []
    y = []
    for i in range(5):
        cur_direct = directory + 'Cluster ' + str(i + 1) + '/'
        for jpg in os.listdir(cur_direct):
            x = ndimage.imread(cur_direct + jpg)
#             x -= np.mean(x)
            patches = extract_patches_2d(x, (img_rows, img_cols), patch_count)
#             print(patches.shape)
            X.extend(patches)
#             print(np.asarray(X).shape)
            # Flatten for simpler models, don't want to do this for CNN
#             X.append((ndimage.imread(cur_direct + jpg, flatten=True)/255.).flatten())
#             X.append(imresize((ndimage.imread(cur_direct + jpg, flatten=False)/255.), (img_rows, img_cols)))
            y.extend([i] * patch_count)
#             print(np.asarray(y).shape)
            
    
    X = np.asarray(X)
    y = np.asarray(y)
    print(X.shape)
    print(y.shape)
    X = X.reshape(X.shape[0], 3, img_rows, img_cols)
    X, y = shuffle(X, y, random_state=seed)  # shuffle train data
    
    X = X.astype('float32')
    y = y.astype('float32')
    return (X, y)

X, y = load()
print(X.shape)
print(y.shape)

(69400, 128, 128, 3)
(69400,)
(69400, 3, 128, 128)
(69400,)


In [3]:
from keras.utils import np_utils
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y, n_folds=3)
y = np_utils.to_categorical(y, 5)

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

nb_classes = 5
accuracy = []
nb_epochs = 100
batch_size = 128

for train, test in skf:    
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same',
                            input_shape=(3, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epochs, show_accuracy=True,
                        validation_data=(X_test, Y_test),
                        nb_worker=1)
    
    score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    accuracy.append(score[1])
print(accuracy)

('X_train shape:', (62460, 3, 128, 128))
(62460, 'train samples')
(6940, 'test samples')
Epoch 1/100
62460/62460 [==============================] - 784s - loss: 16.0634 - acc: 0.9976 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 2/100
62460/62460 [==============================] - 761s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 3/100
62460/62460 [==============================] - 760s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 4/100
62460/62460 [==============================] - 766s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 5/100
62460/62460 [==============================] - 758s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 6/100
62460/62460 [==============================] - 757s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val_acc: 1.0000
Epoch 7/100
62460/62460 [==============================] - 759s - loss: 16.1181 - acc: 1.0000 - val_loss: 16.1144 - val

Using Theano backend.
/home/brett/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


KeyboardInterrupt: 